In [2]:
from random import choice

from game import (
    valid,
    win_condition,
    next_player,
    game_updater,
    get_optimal,
    Board,
    get_expected_values,
    display_likelihood,
)


initial_board, initial_turn, initial_player, initial_winning_condition = game_updater(
    moves=[]
)

likelihood_board: Board = [row[:] for row in initial_board]
initial_moves = valid(initial_board)

for initial_move in initial_moves:
    n = 100
    x = 0
    o = 0
    tie = 0
    turns = []

    for m in range(n):
        board = [row[:] for row in initial_board]
        turn = initial_turn
        player = initial_player
        winning_condition = initial_winning_condition
        while not winning_condition:
            opponent = "O" if player == "X" else "X"
            valid_moves = valid(board)

            winning_moves = get_optimal(board, valid_moves, player)
            blocking_moves = get_optimal(board, valid_moves, opponent)
            optimal_moves = get_optimal(board, valid_moves, player, partial=2)
            defensive_moves = get_optimal(board, valid_moves, opponent, partial=2)

            if len(winning_moves):
                valid_moves = winning_moves
            if len(blocking_moves):
                valid_moves = blocking_moves
            if initial_move and turn == initial_turn:
                valid_moves = [initial_move]
            if len(optimal_moves):
                valid_moves = optimal_moves
            if len(defensive_moves):
                valid_moves = defensive_moves
            if len(valid_moves):
                i, j = choice(valid_moves)
                board[i][j] = player
                winning_condition = win_condition(board, player)
                if winning_condition:
                    if player == "X":
                        x += 1
                    if player == "O":
                        o += 1
                    break
                else:
                    player = next_player(player)
                    if player == "X":
                        turn += 1
            else:
                tie += 1
                winning_condition = "tie"
        turns.append(turn)
    u, v = initial_move
    likelihood_board[u][v] = (x, o, tie)

expected_values = get_expected_values(likelihood_board, initial_player)
print(expected_values[0])
display_likelihood(likelihood_board)

(0.08, 2, 6)


0.020,,,,,,-0.050
0.000,,,,,,0.010
-0.070,,,,,,0.080
-0.330,,,,,,-0.180
-0.060,,,,,,-0.110
0.070,,,,,,0.010
-0.060,,,,,,-0.230
